To-do:
1. Текст был очищен только от одного мусорного элемента в качестве примера. Исследуйте данные через ноутбук или через веб-интерфейс BigQuery на предмет других мусорных элементов в тексте, которые не несут в себе никакого особого смысла, а только создают шум в данных. Доработайте функцию очистки тектосвых данных, чтобы в нее можно было передать список ненужного мусора и разом выполнялась очистка
2. Проведите стратифицировнную кросс-валидацию нейросетевого классификатора https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
3. Реверс-инжениринг нейронной сети (вектор - фразы - частотный анализ встречаемых слов - удаление или добавление в модель)

НЕ НАДО 3. Поэксперементируйте с гиперпараметрами нейросетевого классификатора, постарайтесь повысить качество его работы

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
4. Попробуйте использовать не Word2Vec для получения векторого представления текста, а TF-IDF преобразование http://zabaykin.ru/?p=558 http://nlpx.net/archives/57
5. Попробуйте использовать более тонко настриваемые алгоритмы нейросетей, например из этого видео https://www.youtube.com/watch?v=cPkH1k3U1c8 

In [1]:
import warnings
warnings.filterwarnings('ignore')

from google.oauth2 import service_account
import pandas_gbq 

import numpy as np
import pandas as pd
import math as mt
import datetime as dt

from langdetect import detect
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score
from gensim.models.word2vec import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve,auc
import matplotlib

In [2]:
CREDENTIALS = ''

In [82]:
#funtion for getting fresh data from DWH for workload model
"""[summary]
Funtion for getting fresh data from BigQuery for workload scoring model
[description]
Credentials - google service account object with credentials data for project
[example]
Input: Credentials = credentials_object
Output: description	                                        channel	 category	category_flag
        \nChat transcript:\nVisitor: I want to buy wit...	chat	 ps	        1
        \nChat transcript:\nVisitor: hell i had a prob...	chat	 ps	        1
        \nChat transcript:\nVisitor: لا استطيع الشراء ...	 chat	  ps	     1
"""
def getDwhData(Credentials):
    statement_bigquery_sql = " ".join(["select description, channel, case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 'ps'",
                                       "else 'other'",
                                       "end as category,",
                                       "case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 0",
                                       "else 1",
                                       "end as category_flag",
                                       "from `xsolla_summer_school.customer_support`",
                                       "where manual_category is not null and",
                                       "manual_category <> '' and",
                                       "description is not null and",
                                       "description <> '' and",
                                       "channel is not null and",
                                       "channel <> '' and",
                                       "channel in ('chat','facebook')"])
    
    dataframe_bigquery = pandas_gbq.read_gbq(statement_bigquery_sql,project_id='findcsystem', credentials=Credentials, dialect='standard')

    return dataframe_bigquery


"""[summary]
Function for transform text to lower case
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["text_1","text_2"]
"""
def lowerCase(Corpus):
    corpus = [i.lower().replace('\n',' ') for i in Corpus]
    return corpus

"""[summary]
Function for delete punctuation form text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
################################################Output: ["Text_1","Text_2"]
"""
def clearPunctuation(Corpus):
    corpus = [i.translate(str.maketrans('', '', string.punctuation)) for i in Corpus]
    return corpus

"""[summary]
Function for getting language of text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["en","ru"]
"""
def getTextLanguage(Corpus):
    txt_lang = []
    for txt in Corpus:
        try:
            lang = detect(txt)
            txt_lang.append(lang)
        except:
            lang = 'error'
            txt_lang.append(lang)
    
    return txt_lang


"""[summary]
Function for tokenization text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["word1 word2","word3 word4"]
Output: [["word1","word2"],["word3","word4"]]
"""  
def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 


"""[summary]
Function for clear text after garbage
[description]
Corpus - list or array object, with text data
Substr - string, regular expression
Garbage - list of string
[example]
Input: Corpus = [["word1","word2"],["word3","word4"]]
       Substr = r'word1'
Output: [["word2"],["word3","word4"]]
"""  
def clearTextAfterGarbage(Corpus,Substr):
    clear_corpus = []
    for text in Corpus:
        indexes = []
        text_len = len(text)
        try:
            for i in range(0,text_len):
                res = re.search(Substr,text[i])
                if res != None:
                    indexes.append(i)
                
            #delete garbage word from text
            for index in indexes:
                del text[index]
        
            clear_corpus.append(text)
        except:
            #clear_corpus.append("error")
            clear_corpus.append(text) # а то слишком много ошибок
        
    return clear_corpus

def clearTextAfterGarbageList2(Corpus,Garbage):
    clear_corpus = Corpus
    for Substr in Garbage:
        clear_corpus = clearTextAfterGarbage(clear_corpus,Substr)
    return clear_corpus    

def clearTextAfterGarbageList(Corpus,Garbage):
    clear_corpus = []
    for text in Corpus:
        for Substr in Garbage:
            indexes = []
            text_len = len(text)
            try:
                for i in range(0,text_len):
                    res = re.search(Substr,text[i])
                    if res != None:
                        indexes.append(i)
                
                #delete garbage word from text
                for index in indexes:
                    del text[index]
        
                clear_corpus.append(text)
            except:
                #clear_corpus.append("error")
                clear_corpus.append(text) # а то слишком много ошибок
        
    return clear_corpus

"""[summary]
Function for clear error after clearTextAfterGarbage(Corpus,Substr)
[description]
InsertDataFrame - pandas DataFrame
[example]
Input: InsertDataFrame = [{'description':['Text_1','error','Text_2'], field1':[1,2,3]}]}]
Output: [{'description':['Text_1','Text_2'], field1':[1,3]}]}]
""" 

def clearDescriptionError(InsertDataFrame):
    result = pd.DataFrame()
    result = InsertDataFrame[InsertDataFrame.description != 'error']
    return result

"""[summary]
Build word vector by using pre-trained Word2Vec model
[description]
Size - lenght of vector
Word2Vec_Model - gensim object
"""  
def buildWordVector(Text,Size,Word2Vec_Model):
    vec = np.zeros(Size).reshape((1,Size))
    count = 0.

    for word in Text:
        try:
            vec += Word2Vec_Model[word].reshape((1,Size))
            count += 1.
        except KeyError:
            continue
    
    if count != 0:
        vec /= count
    
    return vec

def countWords(InsertDataFrame):
    df_count = pd.DataFrame({'text':[], 'count':[]})

    for row in InsertDataFrame.description:
        for text in row:            
            if text in df_count.text.values:
                index = df_count[df_count.text == text].index
                df_count.loc[index,'count'] = df_count.loc[index,'count']+1
            else:
                df_count = df_count.append(pd.DataFrame({'text':text, 'count':[int(1)]}))
    return df_count

RAWDATA 

In [4]:
#getting data from dwh
SupportRawDataframe = getDwhData(CREDENTIALS)
SupportRawDataframe.shape
#SupportRawDataframe.head(10)
#SupportRawDataframe.tail(10)

Downloading: 100%|████████████████████| 23450/23450 [00:51<00:00, 454.53rows/s]


(23450, 4)

DATA PREPROC

In [5]:
#transform text to lower case
corpus = SupportRawDataframe.description
corpus.astype('str')

corpus = clearPunctuation( lowerCase(corpus) )

#getting language for text corpus
corpus_lang = getTextLanguage(corpus)

In [6]:
#new dataframe with texts in lower case, without /n symbol and with lang for text
SupportRawDataframe['description'] = corpus
SupportRawDataframe['lang'] = corpus_lang
#SupportRawDataframe.head(10)
#SupportRawDataframe.tail(10)

In [51]:
#getting only en texts
SupportDataframe_eng = SupportRawDataframe[SupportRawDataframe.lang == 'en'][:]
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,chat transcript visitor i want to buy with pa...,chat,ps,0,en
1,chat transcript visitor hell i had a problem ...,chat,ps,0,en
2,chat transcript visitor لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript visitor im having trouble wit...,chat,ps,0,en
4,chat transcript visitor hi ana hello how can ...,chat,ps,0,en
...,...,...,...,...,...
23445,chat transcript visitor hi i made a pruchase ...,chat,other,1,en
23446,chat transcript visitor hi how long will it t...,chat,other,1,en
23447,chat transcript visitor i bought playerunknow...,chat,other,1,en
23448,chat transcript visitor good day i took the w...,chat,other,1,en


In [52]:
#text tekenization
tokenization = textToTokens(SupportDataframe_eng.description)
SupportDataframe_eng['description'] = tokenization

In [53]:
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[chat, transcript, visitor, i, want, to, buy, ...",chat,ps,0,en
1,"[chat, transcript, visitor, hell, i, had, a, p...",chat,ps,0,en
2,"[chat, transcript, visitor, لا, استطيع, الشراء...",chat,ps,0,en
3,"[chat, transcript, visitor, im, having, troubl...",chat,ps,0,en
4,"[chat, transcript, visitor, hi, ana, hello, ho...",chat,ps,0,en
...,...,...,...,...,...
23445,"[chat, transcript, visitor, hi, i, made, a, pr...",chat,other,1,en
23446,"[chat, transcript, visitor, hi, how, long, wil...",chat,other,1,en
23447,"[chat, transcript, visitor, i, bought, playeru...",chat,other,1,en
23448,"[chat, transcript, visitor, good, day, i, took...",chat,other,1,en


In [47]:
texts = list(SupportDataframe_eng.description)

In [87]:
#cleaning text after garbage
GarbageList = ['chat','transcript','visitor', 'hi', 'hello', 'i', 'as', 'for', 'to', 'a', 'the', 'you', 'this', 'or', 'u', 'me', 'hi', 'im', 'your', 'is', 'on', 'its', 'it', 'into']
#tests_clear = clearTextAfterGarbage(texts,r'chat')
#tests_clear = clearTextAfterGarbage(texts,r'transcript')
#tests_clear = clearTextAfterGarbage(texts,r'visitor')
tests_clear = clearTextAfterGarbageList2(texts,GarbageList)

In [88]:
SupportDataframe_eng['description'] = tests_clear
SupportDataframe_eng = clearDescriptionError(SupportDataframe_eng)
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[a, code, a, screenshot, error, an, as, few, t...",chat,ps,0,en
1,"[hell, yesterday, got, i, but, a, 4, gifted, a...",chat,ps,0,en
2,"[لا, استطيع, الشراء, ومعلومات, الشراء, صحيحة, ...",chat,ps,0,en
3,"[4, code, a, ana, we, a, placed, on, 1, twice,...",chat,ps,0,en
4,"[how, pay, card, credit]",chat,ps,0,en
...,...,...,...,...,...
23445,"[25, dollars, my, specified, by, been, i, sent...",chat,other,1,en
23446,"[how, my, friend, contain, with, details, will...",chat,other,1,en
23447,"[refund, 25, received, all, details, alexandra...",chat,other,1,en
23448,"[good, and, way, can, two, resolve, transactio...",chat,other,1,en


In [86]:
'''df_count = pd.DataFrame({'text':[], 'count':[]})
df_count0 = pd.DataFrame({'text':[]})

for row in SupportDataframe_eng.description:
    for text in row:
        df_count0 = df_count0.append(pd.DataFrame({'text':[text]}))               
        if text in df_count.text.values:
            index = df_count[df_count.text == text].index
            df_count.loc[index,'count'] = df_count.loc[index,'count']+1
        else:
            df_count = df_count.append(pd.DataFrame({'text':text, 'count':[int(1)]}))
            
import seaborn as sns
import matplotlib.pyplot as plt

df_count2 = df_count.sort_values('count', ascending=False).iloc[:100,:]
#fig = plt.figure(figsize=(100,10))
#plt.bar(df_count2.text, df_count2['count'])
df_count2.head(50)            
#df_count'''

"df_count = pd.DataFrame({'text':[], 'count':[]})\ndf_count0 = pd.DataFrame({'text':[]})\n\nfor row in SupportDataframe_eng.description:\n    for text in row:\n        df_count0 = df_count0.append(pd.DataFrame({'text':[text]}))               \n        if text in df_count.text.values:\n            index = df_count[df_count.text == text].index\n            df_count.loc[index,'count'] = df_count.loc[index,'count']+1\n        else:\n            df_count = df_count.append(pd.DataFrame({'text':text, 'count':[int(1)]}))\n            \nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\ndf_count2 = df_count.sort_values('count', ascending=False).iloc[:100,:]\n#fig = plt.figure(figsize=(100,10))\n#plt.bar(df_count2.text, df_count2['count'])\ndf_count2.head(50)            \n#df_count"

TRAIN TEST SPLIT DATAFRAMES

In [89]:
#list of unique categories
unique_categories = np.unique(SupportDataframe_eng.category)
unique_categories

array(['other', 'ps'], dtype=object)

In [90]:
descriptions = SupportDataframe_eng['description']
categories = SupportDataframe_eng['category_flag']
XTrain,XTest,YTrain,YTest = train_test_split(descriptions,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)

FROM TEXTS TO VECTORS

In [91]:
#initialize Word2Vec model for embedding words to vectors
NDim = 100
Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
Imdb_w2v.build_vocab(XTrain)

In [92]:
Imdb_w2v.train(XTrain,total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)

(878657, 1304860)

In [93]:
#embedding training messages to vectors for neutral classifier
TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTrain])

In [94]:
TrainVecs

array([[ 0.41770694, -0.13749716, -0.04865682, ...,  0.05241149,
        -0.03612048,  0.10137637],
       [ 0.49398091, -0.00422621,  0.08119957, ...,  0.12447836,
         0.0861813 ,  0.25740374],
       [ 0.38184024, -0.07130834,  0.10378505, ...,  0.2060302 ,
         0.08443201,  0.26127128],
       ...,
       [ 0.47963359,  0.04692873,  0.11349692, ...,  0.21846799,
         0.06216082,  0.14397052],
       [ 0.43374576, -0.06246465,  0.2496388 , ...,  0.15147051,
        -0.10502954,  0.34311489],
       [ 0.47096798,  0.01148131,  0.13035181, ...,  0.16968209,
         0.04469057,  0.14918907]])

In [95]:
Imdb_w2v.train(XTest, total_examples = Imdb_w2v.corpus_count, epochs = Imdb_w2v.epochs)

(220782, 328805)

In [96]:
TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTest])

In [97]:
TestVecs

array([[ 0.21701333, -0.07734585, -0.01326046, ..., -0.01822665,
        -0.10563625,  0.15623976],
       [ 0.40639109,  0.09373962,  0.15100237, ...,  0.12765931,
         0.04488374,  0.11340171],
       [ 0.60968511,  0.3570559 ,  0.42759349, ...,  0.38786559,
        -0.13404563,  0.04923485],
       ...,
       [ 0.44847175,  0.07205314, -0.44483176, ...,  0.37589905,
         0.47444311, -0.08229289],
       [ 0.30208631, -0.06710074, -0.06477036, ...,  0.11963807,
         0.15459059,  0.06940154],
       [ 0.29998545, -0.14695022, -0.33885496, ..., -0.27240485,
         0.02898898,  0.13579771]])

CROSSVALIDATION AND BUILD CLASSIFIER

In [98]:
TextClassifier = MLPClassifier(hidden_layer_sizes = (20,10), max_iter = 1000, random_state = 40)

In [99]:
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = 5)
print(Scores)
print(np.mean(Scores))

[0.82423539 0.83894696 0.85127808 0.84314485 0.82920217]
0.8373614885866685


In [100]:
TextClassifier.fit(TrainVecs,YTrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=40, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [101]:
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[ 283  367]
 [ 184 2395]]
              precision    recall  f1-score   support

           0       0.61      0.44      0.51       650
           1       0.87      0.93      0.90      2579

    accuracy                           0.83      3229
   macro avg       0.74      0.68      0.70      3229
weighted avg       0.81      0.83      0.82      3229



Кросс-валидация

In [102]:
skf = StratifiedKFold(n_splits=2, random_state=40, shuffle=False)
MLPClassifier_sum = int(0)

NDim = 100

for train_index, test_index in skf.split(descriptions, categories):
    Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
    Imdb_w2v.build_vocab(descriptions.loc[descriptions.index.intersection(train_index)])
    Imdb_w2v.train(descriptions.loc[descriptions.index.intersection(train_index)],total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)
    TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in descriptions.loc[descriptions.index.intersection(train_index)]])
    Imdb_w2v.train(descriptions.loc[descriptions.index.intersection(test_index)], total_examples = Imdb_w2v.corpus_count, epochs = Imdb_w2v.epochs)
    TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in descriptions.loc[descriptions.index.intersection(test_index)]])
    
    TextClassifier.fit(TrainVecs,categories.loc[categories.index.intersection(train_index)])
    MLPClassifier_sum += roc_auc_score(categories.loc[categories.index.intersection(test_index)], TextClassifier.predict(TestVecs))
    print(classification_report(categories.loc[categories.index.intersection(test_index)],TextClassifier.predict(TestVecs)))
MLPClassifier_total = MLPClassifier_sum / skf.get_n_splits()
print('mean roc_auc', MLPClassifier_total)

              precision    recall  f1-score   support

           0       0.78      0.11      0.20      2389
           1       0.61      0.98      0.75      3341

    accuracy                           0.62      5730
   macro avg       0.69      0.55      0.47      5730
weighted avg       0.68      0.62      0.52      5730

              precision    recall  f1-score   support

           0       0.26      0.89      0.40       861
           1       0.96      0.52      0.68      4563

    accuracy                           0.58      5424
   macro avg       0.61      0.71      0.54      5424
weighted avg       0.85      0.58      0.63      5424

mean roc_auc 0.6252041094208673


Зависит ли точность модели от симметричности категорий рандомной выборки?

Но тут рассмотрим слова в ошибочно отнесённых записях

In [103]:
# index of wrong predictions
error_zero = list()
error_one = list()
for i in range(YTest.size):
    if YTest.iloc[i] == 0:
        if YTest.iloc[i] != pred[i]:
            error_zero.append(YTest.index[i])
    else:
        if YTest.iloc[i] != pred[i]:
            error_one.append(YTest.index[i])        

In [104]:
df_error_zero = SupportDataframe_eng[SupportDataframe_eng.index.isin(error_zero)]
df_error_one = SupportDataframe_eng[SupportDataframe_eng.index.isin(error_one)]

In [105]:
words_error_zero = countWords(df_error_zero)
words_error_one = countWords(df_error_one)

In [106]:
words_error_zero.head(50)

,text,count
0,msg,5437.0
0,browser,5437.0
0,out,5437.0
0,allow,5437.0
0,that,5437.0
0,does,5437.0
0,the,5437.0
0,as,5437.0
0,why,5437.0
0,we,5437.0


In [107]:
words_error_one.head(50)

,text,count
0,2100,2220.0
0,sky,2220.0
0,no,2220.0
0,be,2219.0
0,ian,2219.0
0,my,2219.0
0,possibly,2219.0
0,did,2219.0
0,may,2218.0
0,could,2218.0
